In [1]:
import mesa
import numpy as np

In [72]:
def randomStrategy(hand, current_agent, other_agents):
    missing_cards = hand.missingCards(flat = True, exclude_unowned_groups = True)
    random_card = np.random.choice(missing_cards)
    other_agent = np.random.choice(other_agents)
    return random_card, other_agent

In [70]:
def flatten(l):
    return [item for sublist in l for item in sublist]

class Hand:
    def __init__(self, model):
        self.model = model
        self.quartet_groups = [QuartetGroup(quartet_set.group_name, quartet_set.card_names) 
                              for quartet_set in self.model.quartet_sets]
    
    def getCards(self, flat = False):
        cards = [quarted_group.getCards() for quarted_group in self.quartet_groups]
        if flat:
            cards = flatten(cards)
        return cards
    
    def addCard(self, new_card):
        new_card_group_name = new_card.group_name
        for quartet_group in self.quartet_groups:
            if quartet_group.group_name == new_card_group_name:
                quartet_group.addCard(new_card)
                return quartet_group.isComplete()
    
    def missingCards(self, flat = False, exclude_unowned_groups = False):
        missing_cards = [group.missingCards() for group in self.quartet_groups]
        if exclude_unowned_groups:
            new_missing_cards = []
            for missing_cards_group in missing_cards:
                if len(missing_cards_group) != 4:
                    new_missing_cards.append(missing_cards_group)
            missing_cards = new_missing_cards
                    
        if flat:
            missing_cards = flatten(missing_cards)
        return missing_cards
    
    def getGroup(self, group_name):
        for quartet_group in self.quartet_groups:
            if quartet_group.group_name == group_name:
                return quartet_group
    
    def getCard(self, asked_card):
        quartet_group = self.getGroup(asked_card.group_name)
        for card in quartet_group.cards:
            if card.card_name == asked_card.card_name:
                return card
    
    def giveCard(self, asked_card):
        card = self.getCard(asked_card)
        if card.owned:
            card.owned = False
            return True
        return False

class QuartetGroup:
    def __init__(self, group_name, card_names):
        self.group_name = group_name
        self.card_names = card_names
        self.cards = [QuartetCard(group_name, card_name, self) for card_name in card_names]
    
    def addCard(self, new_card):
        for card in self.cards:
            if new_card == card:
                card.owned = True
    
    def missingCards(self):
        current_cards = self.getCards()
        return [x for x in self.cards if x not in current_cards]
    
    def getCards(self):
        return [x for x in self.cards if x.owned]
    
    def isComplete(self):
        return np.all([x.owned for x in self.cards])
    
    def __repr__(self):
        return str([card for card in self.cards])

class QuartetCard:
    def __init__(self, group_name, card_name, quartet_group, owned = False):
        self.group_name = group_name
        self.card_name = card_name
        self.quartet_group = quartet_group
        self.owned = owned
    
    def __eq__(self, other): 
        if not isinstance(other, QuartetCard):
            return NotImplemented

        return self.group_name == other.group_name and self.card_name == other.card_name
    
    def __repr__(self):
        return self.group_name + "_" + self.card_name
    
    def __hash__(self):
        return hash((self.group_name, self.card_name))

class QuartetAgent(mesa.Agent):
    
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.hand = Hand(model)
        self.score = 0
        self.strategy = randomStrategy
        self.other_agents = [i for i in range(self.model.num_agents) if i != self.unique_id]
        
    def step(self):
        print("I am agent", self.unique_id)
        print("I currently have these cards:")
        print(self.hand.getCards())
        print("I currently do not have these cards:")
        print(self.hand.missingCards())
        
        asked_card, other_agent = self.strategy(self.hand, self.unique_id, self.other_agents)
        got_card = self.model.askForCard(self.unique_id, other_agent, asked_card)
        if got_card:
            self.addCard(asked_card)
            print("Now, I have these cards:")
            print(self.hand.getCards())
    
    def addCard(self, new_card):
        has_complete_set = self.hand.addCard(new_card)
        if has_complete_set:
            pass
            
    
    def giveCard(self, card):
        return self.hand.giveCard(card)
        
    

class QuartetModel(mesa.Model):
    
    def __init__(self, N):
        self.num_agents = N
        self.schedule = mesa.time.BaseScheduler(self)
        self.quartet_sets = [QuartetGroup('a', ['a_1', 'a_2', 'a_3', 'a_4']), 
                             QuartetGroup('b', ['b_1', 'b_2', 'b_3', 'b_4']), 
                             QuartetGroup('c', ['c_1', 'c_2', 'c_3', 'c_4'])]
        # Create agents
        for i in range(self.num_agents):
            a = QuartetAgent(i, self)
            self.schedule.add(a)
        
        all_cards = []
        for quartet_set in self.quartet_sets:
            for quartet_card in quartet_set.cards:
                all_cards.append(quartet_card)
        np.random.shuffle(all_cards)
        
        cards_per_agent = 4
        for i in range(cards_per_agent):
            for agent in self.schedule.agents:
                current_card = all_cards.pop()
                agent.addCard(current_card)
                

    def step(self):
        """Advance the model by one step."""
        self.schedule.step()
    
    def askForCard(self, asking_agent, asked_agent, card):
        print("PA: Agent", asking_agent, "asks agent", asked_agent, "for card", card)
        asked_agent_object = self.schedule.agents[asked_agent]
        has_card = asked_agent_object.giveCard(card)
        if has_card:
            print("PA: agent", asked_agent, "had the card and gave it to", asking_agent)
        else:
            print("PA: agent", asked_agent, "did not have the card.")
        return has_card

In [71]:
model = QuartetModel(3)

model.step()

I am agent 0
I currently have these cards:
[[a_a_1], [b_b_1, b_b_2, b_b_4], []]
I currently do not have these cards:
[[a_a_2, a_a_3, a_a_4], [b_b_3], [c_c_1, c_c_2, c_c_3, c_c_4]]
potential cards to ask: [a_a_2, a_a_3, a_a_4, b_b_3]
PA: Agent 0 asks agent 2 for card b_b_3
PA: agent 2 did not have the card.
I am agent 1
I currently have these cards:
[[a_a_4], [b_b_3], [c_c_2, c_c_3]]
I currently do not have these cards:
[[a_a_1, a_a_2, a_a_3], [b_b_1, b_b_2, b_b_4], [c_c_1, c_c_4]]
potential cards to ask: [a_a_1, a_a_2, a_a_3, b_b_1, b_b_2, b_b_4, c_c_1, c_c_4]
PA: Agent 1 asks agent 0 for card a_a_1
PA: agent 0 had the card and gave it to 1
Now, I have these cards:
[[a_a_1, a_a_4], [b_b_3], [c_c_2, c_c_3]]
I am agent 2
I currently have these cards:
[[a_a_2, a_a_3], [], [c_c_1, c_c_4]]
I currently do not have these cards:
[[a_a_1, a_a_4], [b_b_1, b_b_2, b_b_3, b_b_4], [c_c_2, c_c_3]]
potential cards to ask: [a_a_1, a_a_4, c_c_2, c_c_3]
PA: Agent 2 asks agent 0 for card c_c_3
PA: agent 0